# Cassava Leaf Disease Classification

Instantiates the Xception architecture.

In [1]:
 !pip uninstall kaggle
 !pip install -U kaggle

Uninstalling kaggle-1.5.10:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.10.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.10
     |████████████████████████████████| 61kB 8.4MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp36-none-any.whl size=73269 sha256=50a80df5957cd8e62192b1783747f21b96347f466896a4996c14949e90611c08
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle


In [2]:
  from google.colab import files
  uploaded = files.upload()

  !pip install -U kaggle

  !mkdir ~/.kaggle

  !cp kaggle.json ~/.kaggle/

  !chmod 600 ~/.kaggle/kaggle.json

  !kaggle competitions download -c cassava-leaf-disease-classification

  !mkdir cassava-leaf-disease-classification

  !unzip cassava-leaf-disease-classification.zip -d  cassava-leaf-disease-classification

  print("\nGet Dataset of Keras/Xception:\n")

  !kaggle datasets download -d keras/xception

  !unzip xception.zip -d "/content/xception" 

  print("Xception zip file converted ! ")
  print("\nRemove the zip files:")
  
  !rm *.zip

  print("Converting Completed..")

Streaming output truncated to the last 5000 lines.
  inflating: cassava-leaf-disease-classification/train_images/3956550570.jpg  
  inflating: cassava-leaf-disease-classification/train_images/3956605397.jpg  
  inflating: cassava-leaf-disease-classification/train_images/3956778160.jpg  
  inflating: cassava-leaf-disease-classification/train_images/3957562076.jpg  
  inflating: cassava-leaf-disease-classification/train_images/3957612771.jpg  
  inflating: cassava-leaf-disease-classification/train_images/3957807023.jpg  
  inflating: cassava-leaf-disease-classification/train_images/3957819631.jpg  
  inflating: cassava-leaf-disease-classification/train_images/395820132.jpg  
  inflating: cassava-leaf-disease-classification/train_images/3958304403.jpg  
  inflating: cassava-leaf-disease-classification/train_images/395852128.jpg  
  inflating: cassava-leaf-disease-classification/train_images/395853907.jpg  
  inflating: cassava-leaf-disease-classification/train_images/3958714206.jpg  
  in

**Listing competitions**

In [4]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        109           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        220           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        437           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2972           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      23317            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       6230           False  
connectx

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
import warnings
import re

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adamax
from keras.models import Sequential

import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

print('Tensorflor:', tf.__version__)

Tensorflor: 2.4.0


#Check GPU state

In [5]:
from tensorflow.python.framework.config import set_memory_growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [6]:

from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
Compute dtype: float16
Variable dtype: float32


**Downloading a dataset**

In [26]:
img_width = 300
img_height = 300
BATCH_SIZE = 32

#seed=64

DROPOUT_RATE = 0.2 #--> Layer olarak eklenecek.

epochs =40;
EPOCHS =40;
last_epoch=0;
validation_split = 0.2

#Xception_NOTOP = '/content/xception/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'

general_path = '/content/cassava-leaf-disease-classification/'

In [27]:

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 0
seed_everything(seed)
warnings.filterwarnings('ignore')


# Read train data

In [28]:
train = pd.read_csv(general_path + 'train.csv')
train['label'] = train['label'].astype('string')
display(train.head())

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [29]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [30]:
train_datagen = ImageDataGenerator(
                            horizontal_flip=True,
                            vertical_flip=True,
                            rotation_range=20,
                            shear_range=20,
                            zoom_range=0.3,  #REVIZE                            
                            fill_mode='nearest',
                            height_shift_range=0.1,
                            width_shift_range=0.1,                            
                            validation_split=validation_split)

train_datagen_flow = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory=general_path + 'train_images',
    x_col='image_id',
    y_col='label',
    target_size=(img_width, img_height),
    batch_size=BATCH_SIZE,
    seed=seed,
    shuffle=True,
    subset='training',
    class_mode='sparse',
    validate_filenames=False)

Found 17118 non-validated image filenames belonging to 5 classes.


In [31]:
valid_datagen = keras.preprocessing.image.ImageDataGenerator( validation_split=validation_split)

valid_datagen_flow = valid_datagen.flow_from_dataframe(
    dataframe=train,
    directory=general_path + 'train_images',
    x_col='image_id',
    y_col='label',
    target_size=(img_width, img_height),
    batch_size=BATCH_SIZE,
    seed=seed,
    shuffle=True,
    subset='validation',
    class_mode='sparse',
    validate_filenames=False)

Found 4279 non-validated image filenames belonging to 5 classes.


# Callback

1. ModelCheckpoint : Callback to save the Keras model or model weights at some frequency.

In [32]:
model_checkpoint = keras.callbacks.ModelCheckpoint(
    './best_weights.h5',
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="min"
)

2. EarlyStopping : Stop training when a monitored metric has stopped improving.

In [33]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=5,
    verbose=1,
    mode="min",
    restore_best_weights=True,
)

3. ReduceLROnPlateau : Reduce learning rate when a metric has stopped improving.

In [34]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=2,
    verbose=0,
    mode="min",
    min_delta=0.001,
)

4.Stream the epoch loss to a file in JSON format. The file content is not well-formed JSON but rather has a JSON object per line.

In [35]:
import json
json_log = open(general_path+'/loss_log.json', mode='w', buffering=1)
json_logging_callback = keras.callbacks.LambdaCallback(
    on_train_begin=lambda logs: json_log.write('{ "train": [ \n'),
    on_epoch_end=lambda epoch,logs: json_log.write(json.dumps({'epoch': epoch, 'loss': logs['loss'], 'acc':logs['accuracy']}) + ',\n'),
    on_train_end=lambda logs: json_log.write(']\n}'),
)

#Transfer learning and fine-tuning

The most common incarnation of transfer learning in the context of deep learning is the following workflow:

1.   Take layers from a previously trained model.

2.   Freeze them, so as to avoid destroying any of the information they contain 
during future training rounds.
3.   Add some new, trainable layers on top of the frozen layers. They will learn to turn the old features into predictions on a new dataset.
4.   Train the new layers on your dataset.

####  Create base model

In [16]:
data_augmentation = keras.Sequential(
    [
        #layers.experimental.preprocessing.RandomCrop(height=img_height, width=img_width),
        layers.experimental.preprocessing.CenterCrop(height=img_height, width=img_width),
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomRotation(0.25),
        layers.experimental.preprocessing.RandomZoom((0.2, 0)),
        layers.experimental.preprocessing.RandomContrast((0.2,0.2))
    ]
)

In [17]:
# Create base model
base_model = keras.applications.EfficientNetB3(
    weights='imagenet', # Load weights pre-trained on ImageNet.
    input_shape=(img_height, img_width, 3),
    include_top=False) # Do not include the ImageNet classifier at the top.

# Freeze base model
base_model.trainable = False

#model.summary()

43941888/43941136 [==============================] - 0s 0us/step


####  Create new model on top.

In [18]:
# Create new model on top.
inputs = keras.Input(shape=(img_height, img_width, 3))
x = data_augmentation(inputs)  # Apply random data augmentation
x = base_model(inputs, training=False)

# The base model contains batchnorm layers. 
# We want to keep them in inference mode when we unfreeze the base model for fine-tuning, so we make sure that the base_model is running in inference mode here.
x = keras.layers.GlobalAveragePooling2D()(x)

# Regularize with dropout
x = keras.layers.Dropout(0.2)(x) 

outputs = keras.layers.Dense(5, activation='softmax', dtype='float32', name='predictions')(x)

model = keras.Model(inputs, outputs)

  **Print model summary**

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, 10, 10, 1536)      10783535  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
predictions (Dense)          (None, 5)                 7685      
Total params: 10,791,220
Trainable params: 7,685
Non-trainable params: 10,783,535
_________________________________________________________________


####  Compile 

In [20]:
model.compile(
    optimizer=keras.optimizers.Adamax(1e-5),  # Low learning rate -- It means 1 × 10−5. In other words, 0.00001
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.AUTO, name='sparse_categorical_crossentropy'),
    metrics=['sparse_categorical_accuracy','accuracy'],
)


  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)



In [21]:
print("Started Training...")

# Trains the model for a fixed number of epochs (iterations on a dataset).
history = model.fit(
    train_datagen_flow,
    epochs=epochs,
    steps_per_epoch=(len(train)*(1-validation_split)) // BATCH_SIZE,
    validation_data=valid_datagen_flow,
    validation_steps=(len(train)*validation_split) // BATCH_SIZE,
    callbacks = [ 
        model_checkpoint, 
        #early_stopping, 
        reduce_lr,
        json_logging_callback
    ],
    use_multiprocessing=False,
    verbose=1
)
print("Training completed")

Started Training...
Epoch 1/10
534/534 [==============================] - 495s 901ms/step - loss: 1.6620 - sparse_categorical_accuracy: 0.2190 - accuracy: 0.2190 - val_loss: 1.4539 - val_sparse_categorical_accuracy: 0.4490 - val_accuracy: 0.4490

Epoch 00001: val_loss improved from inf to 1.45390, saving model to ./best_weights.h5
Epoch 2/10
534/534 [==============================] - 450s 843ms/step - loss: 1.3304 - sparse_categorical_accuracy: 0.5524 - accuracy: 0.5524 - val_loss: 1.2783 - val_sparse_categorical_accuracy: 0.5989 - val_accuracy: 0.5989

Epoch 00002: val_loss improved from 1.45390 to 1.27829, saving model to ./best_weights.h5
Epoch 3/10
534/534 [==============================] - 450s 843ms/step - loss: 1.1910 - sparse_categorical_accuracy: 0.6172 - accuracy: 0.6172 - val_loss: 1.1973 - val_sparse_categorical_accuracy: 0.6147 - val_accuracy: 0.6147

Epoch 00003: val_loss improved from 1.27829 to 1.19731, saving model to ./best_weights.h5
Epoch 4/10
534/534 [=============

###  **Train the top layer**

#### Top Model Compile

In [23]:
# Unfreeze the base_model. Note that it keeps running in inference mode since we passed `training=False` when calling it.
# This means that the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training we've done so far.

base_model.trainable = True

model.compile(
    optimizer=keras.optimizers.Adamax(lr=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.AUTO, name='sparse_categorical_crossentropy'),
    metrics=['sparse_categorical_accuracy','accuracy'],
)


  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)



#### Start Training

We have defined everything we need, it's time to train the model...

In [ ]:
print("Started Training...")

# Trains the model for a fixed number of epochs (iterations on a dataset).
history = model.fit(
    train_datagen_flow,
    epochs=epochs,
    steps_per_epoch=(len(train)*(1-validation_split)) // BATCH_SIZE,
    validation_data=valid_datagen_flow,
    validation_steps=(len(train)*validation_split) // BATCH_SIZE,
    callbacks = [ 
        model_checkpoint, 
        reduce_lr,
        json_logging_callback,
        early_stopping
    ],
    use_multiprocessing=False,
    verbose=1
)
print("Training completed")

# Performance
Since the performance metric for this competition is accuracy, let us plot the train and validation accuracy to monitor our model performance.

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'])
plt.ylim(ymax = 2, ymin = 0)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('Sparse Categorical Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Sparse Categorical Accuracy')
plt.legend(['Train','Test'])
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train','Test'])
plt.grid()
plt.show()

# Submission

In [ ]:
submission = pd.DataFrame(columns=['image_id','label'])
for image_name in os.listdir(general_path + 'test_images'):
    image_path = os.path.join(general_path + 'test_images', image_name)
    image = tf.keras.preprocessing.image.load_img(image_path)
    resized_image = image.resize((img_width, img_height))
    numpied_image = np.expand_dims(resized_image, 0)
    tensored_image = tf.cast(numpied_image, tf.float32)
    submission = submission.append(pd.DataFrame({'image_id': image_name,
                                                 'label': model.predict_classes(tensored_image)}))

submission.to_csv('/content/submission.csv', index=False)

In [ ]:
model.save('/content/TrainedModel/ModelSaved')

In [ ]:
"""
model = keras.models.Sequential()
model.add(keras.applications.EfficientNetB3(
    input_shape=(img_height, img_width, 3), 
    weights='imagenet', 
    include_top=False)
)

model.add(keras.layers.GlobalAveragePooling2D())
model.add(layers.Dropout(DROPOUT_RATE))
#model.add(layers.BatchNormalization())
model.add(keras.layers.Dense(5, activation='softmax', dtype='float32', name='predictions'))
"""

**Receiver Operating Characteristic (ROC)**


**AUC: Area Under the ROC Curve**

---

